# Softmax Classifier

### 10 labels => 10 outputs

### [Softmax function](https://www.ritchieng.com/machine-learning/deep-learning/neural-nets/)

We can turn scores (logits) into probabilities using a softmax function.
- The probabilities will all sum to 1.
- Probability will be low if it's a high score.
- Probability will be high if it's a low score.

$$ \sigma(z)_j = { e^{z_j} \over \sum^K_{k=1} e^{z_k}} $$ for $j = 1, \cdots, K$

<img src="https://camo.githubusercontent.com/a443fe7e4e78e6f7cda8c14d2c33cc69c51d732e/687474703a2f2f692e696d6775722e636f6d2f4b4f78375338472e706e67" width=500>

 ### Cost function : Cross entropy
 
 $$ D(\hat{Y},Y) = -Y \log \hat{Y} $$
    

### Loss Function
$$ L = {1\over N}\sum_i D(S(w*x_i+b),y_i)  $$

In [6]:
import numpy as np

Y = np.array([1, 0, 0])

Y_pred1 = np.array([0.7, 0.2, 0.1])
Y_pred2 = np.array([0.1, 0.3, 0.6])

print('loss1 = ', np.sum(-Y * np.log(Y_pred1)))
print('loss2 = ', np.sum(-Y * np.log(Y_pred2)))

loss1 =  0.35667494393873245
loss2 =  2.3025850929940455


In [10]:
import torch
from torch.autograd import Variable

loss = torch.nn.CrossEntropyLoss()

Y = Variable(torch.LongTensor([0]), requires_grad=False)

Y_pred1 = Variable(torch.Tensor([[2.0, 1.0, 0.1]]))
Y_pred2 = Variable(torch.Tensor([[0.5, 2.0, 0.3]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print('Pytorch Loss1 = ', l1.data, 
      '\nPytorch Loss2 = ', l2.data)


Pytorch Loss1 =  tensor(0.4170) 
Pytorch Loss2 =  tensor(1.8406)


In [14]:
# target is of size nBatch
# each element in target has to have 0 <= value < nClasses (0-2)
# Input is class, not one-hot
Y = Variable(torch.LongTensor([2, 0, 1]), requires_grad=False)

# input is of size nBatch x nClasses = 2 x 4
# Y_pred are logits (not softmax)
Y_pred1 = Variable(torch.Tensor([[0.1, 0.2, 0.9],
                                 [1.1, 0.1, 0.2],
                                 [0.2, 2.1, 0.1]]))


Y_pred2 = Variable(torch.Tensor([[0.8, 0.2, 0.3],
                                 [0.2, 0.3, 0.5],
                                 [0.2, 0.2, 0.5]]))

l1 = loss(Y_pred1, Y)
l2 = loss(Y_pred2, Y)

print("Batch Loss1 =", l1.data, "\nBatch Loss2 =", l2.data)

Batch Loss1 = tensor(0.4966) 
Batch Loss2 = tensor(1.2389)


## Exercise 9-1 : CrossEntropyLoss vs NLLLoss

- What are the differnces?
- Check out
 - https://pytorch.org/docs/master/nn.html#nllloss
 - https://pytorch.org/docs/master/nn.html#crossentropyloss
- Minimizing the Negative Log-Likelihood in English
  (http://willwolf.io/2017/05/18/minimizing_the_negative_log_likelihood_in_english/)

## NMIST Network

http://ml4a.github.io/ml4a/ko/looking_inside_neural_nets/


In [22]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

# Training settings
batch_size = 64

# MNIST Dataset
train_dataset = datasets.MNIST(root='./data/',
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root='./data/',
                              train=False,
                              transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x)


model = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)


def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 200 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))


def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data[0]
        # get the index of the max
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


for epoch in range(1, 10):
    train(epoch)
    test()

/Users/Jun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:70: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.301587
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.289120
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.291083
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.245781
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.952906


/Users/Jun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:78: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/Jun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:81: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number



Test set: Average loss: 0.0177, Accuracy: 6962/10000 (69%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.103479
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.872082
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.803695
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.476803
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.496327

Test set: Average loss: 0.0063, Accuracy: 8835/10000 (88%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.405531
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.320144
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.475870
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.321615
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.266144

Test set: Average loss: 0.0047, Accuracy: 9157/10000 (91%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.190647
Train Epoch: 4 [12800/60000 (21%)]	Loss: 0.229722
Train Epoch: 4 [25600/60000 (43%)]	Loss: 0.152370
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.316723
Train Epoch: 4 [51200/60000 (85%)]	Loss: 0.175720

Test set: Average loss: 0.0034, Accuracy: 9361/10000 (93%)

Train Epoch: 5 [0/600